Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import pickle as pkl

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f, encoding='latin1')
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [8]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('-------------------------------')
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
-------------------------------
Loss at step 0: 17.453949
Training accuracy: 14.3%
Validation accuracy: 16.0%
-------------------------------
Loss at step 100: 2.355172
Training accuracy: 72.0%
Validation accuracy: 70.5%
-------------------------------
Loss at step 200: 1.901910
Training accuracy: 74.9%
Validation accuracy: 72.5%
-------------------------------
Loss at step 300: 1.654243
Training accuracy: 76.2%
Validation accuracy: 73.4%
-------------------------------
Loss at step 400: 1.484323
Training accuracy: 77.1%
Validation accuracy: 74.0%
-------------------------------
Loss at step 500: 1.357427
Training accuracy: 77.7%
Validation accuracy: 74.4%
-------------------------------
Loss at step 600: 1.257688
Training accuracy: 78.3%
Validation accuracy: 74.6%
-------------------------------
Loss at step 700: 1.176400
Training accuracy: 78.7%
Validation accuracy: 74.9%
-------------------------------
Loss at step 800: 1.108428
Training accuracy: 79.3%
Validation accura

Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [16]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [19]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('-------------------------------')  
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
offset:	0
step:	0
batch_size:	0
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
-------------------------------
Minibatch loss at step 0: 17.652439
Minibatch accuracy: 4.7%
Validation accuracy: 13.6%
offset:	128
step:	1
batch_size:	128
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	256
step:	2
batch_size:	256
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	384
step:	3
batch_size:	384
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	512
step:	4
batch_size:	512
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	640
step:	5
batch_size:	640
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	768
step:	6
batch_size:	768
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1

offset:	15104
step:	118
batch_size:	15104
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	15232
step:	119
batch_size:	15232
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	15360
step:	120
batch_size:	15360
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	15488
step:	121
batch_size:	15488
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	15616
step:	122
batch_size:	15616
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	15744
step:	123
batch_size:	15744
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	15872
step:	124
batch_size:	15872
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	16000
step:	125
batch_size:	16000
train_labels.shape[0]:	(200000, 10)
train

offset:	25472
step:	199
batch_size:	25472
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	25600
step:	200
batch_size:	25600
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	25728
step:	201
batch_size:	25728
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	25856
step:	202
batch_size:	25856
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	25984
step:	203
batch_size:	25984
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	26112
step:	204
batch_size:	26112
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	26240
step:	205
batch_size:	26240
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	26368
step:	206
batch_size:	26368
train_labels.shape[0]:	(200000, 10)
train

offset:	34816
step:	272
batch_size:	34816
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	34944
step:	273
batch_size:	34944
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	35072
step:	274
batch_size:	35072
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	35200
step:	275
batch_size:	35200
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	35328
step:	276
batch_size:	35328
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	35456
step:	277
batch_size:	35456
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	35584
step:	278
batch_size:	35584
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	35712
step:	279
batch_size:	35712
train_labels.shape[0]:	(200000, 10)
train

train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	44416
step:	347
batch_size:	44416
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	44544
step:	348
batch_size:	44544
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	44672
step:	349
batch_size:	44672
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	44800
step:	350
batch_size:	44800
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	44928
step:	351
batch_size:	44928
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	45056
step:	352
batch_size:	45056
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	45184
step:	353
batch_size:	45184
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	45312
step:	354
batch_

...

step:	2654
batch_size:	139840
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	139968
step:	2655
batch_size:	139968
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	140096
step:	2656
batch_size:	140096
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	140224
step:	2657
batch_size:	140224
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	140352
step:	2658
batch_size:	140352
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	140480
step:	2659
batch_size:	140480
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	140608
step:	2660
batch_size:	140608
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	140736
step:	2661
batch_size:	140736
train_labels.shape[0]:	(200000, 

offset:	151744
step:	2747
batch_size:	151744
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	151872
step:	2748
batch_size:	151872
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	152000
step:	2749
batch_size:	152000
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	152128
step:	2750
batch_size:	152128
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	152256
step:	2751
batch_size:	152256
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	152384
step:	2752
batch_size:	152384
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	152512
step:	2753
batch_size:	152512
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	152640
step:	2754
batch_size:	152640
train_labels.shap

offset:	161984
step:	2827
batch_size:	161984
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	162112
step:	2828
batch_size:	162112
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	162240
step:	2829
batch_size:	162240
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	162368
step:	2830
batch_size:	162368
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	162496
step:	2831
batch_size:	162496
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	162624
step:	2832
batch_size:	162624
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	162752
step:	2833
batch_size:	162752
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	162880
step:	2834
batch_size:	162880
train_labels.shap

train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	172864
step:	2912
batch_size:	172864
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	172992
step:	2913
batch_size:	172992
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	173120
step:	2914
batch_size:	173120
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	173248
step:	2915
batch_size:	173248
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	173376
step:	2916
batch_size:	173376
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	173504
step:	2917
batch_size:	173504
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	173632
step:	2918
batch_size:	173632
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	1

offset:	182720
step:	2989
batch_size:	182720
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	182848
step:	2990
batch_size:	182848
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	182976
step:	2991
batch_size:	182976
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	183104
step:	2992
batch_size:	183104
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	183232
step:	2993
batch_size:	183232
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	183360
step:	2994
batch_size:	183360
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	183488
step:	2995
batch_size:	183488
train_labels.shape[0]:	(200000, 10)
train_label[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
offset:	183616
step:	2996
batch_size:	183616
train_labels.shap

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [15]:
batch_size = 128
num_nodes= 1024

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases1 = tf.Variable(tf.zeros([num_nodes]))
    weights2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    logit2 = tf.matmul(layer1, weights2) + biases2
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logit2))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    layer1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    test_prediction = tf.nn.relu(tf.matmul(layer1, weights2) + biases2)
    
    layer1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    valid_prediction = tf.nn.relu(tf.matmul(layer1, weights2) + biases2)

In [21]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('-------------------------------')  
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
-------------------------------
Minibatch loss at step 0: 16.377649
Minibatch accuracy: 7.8%
Validation accuracy: 11.2%
Test accuracy: 11.5%
-------------------------------
Minibatch loss at step 500: 1.203869
Minibatch accuracy: 77.3%
Validation accuracy: 75.9%
Test accuracy: 83.4%
-------------------------------
Minibatch loss at step 1000: 1.452421
Minibatch accuracy: 73.4%
Validation accuracy: 76.8%
Test accuracy: 84.3%
-------------------------------
Minibatch loss at step 1500: 0.760515
Minibatch accuracy: 83.6%
Validation accuracy: 77.1%
Test accuracy: 84.9%
-------------------------------
Minibatch loss at step 2000: 0.829501
Minibatch accuracy: 78.9%
Validation accuracy: 77.5%
Test accuracy: 85.3%
-------------------------------
Minibatch loss at step 2500: 0.960108
Minibatch accuracy: 78.1%
Validation accuracy: 78.4%
Test accuracy: 85.9%
-------------------------------
Minibatch loss at step 3000: 1.044487
Minibatch accuracy: 76.6%
Validation accuracy: 78.7%
Test 